# Pre-Photometry before measuring with GALFITM

In [53]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from astropy.io import fits

In [87]:
img_name = 'nircam6'
img_dir = f'../data/image/{img_name}/'
img_merged_path = f'./io/image/{img_name}/merged.fits'

match_dir = f'./io/match/{img_name}/'
prep_dir = f'./io/prep/{img_name}/'

## Create Source Catalog

In [88]:
from photutils.segmentation import SegmentationImage
from photutils.segmentation import SourceCatalog
from astropy.wcs import WCS
from astropy.table import Table, hstack

In [89]:
with fits.open(img_merged_path) as hdul:
    segm = SegmentationImage(hdul['SEGMAP'].data)
    w = WCS(hdul['MERGED'].header)
    image_merged_conv = hdul['SCI_BKSUB_CONV'].data

In [90]:
tab_ref_matched_selected = Table.read(match_dir + 'matchlog.csv', format='ascii.csv')
tab_sorted = tab_ref_matched_selected[tab_ref_matched_selected.argsort('label')] # sort by label
tab_sorted = tab_sorted[tab_sorted['label'] != -1]

label_column = np.int64(tab_sorted['label'])
id_column = np.array(tab_sorted['ID'])
ra_column = tab_sorted['RA']
dec_column = tab_sorted['Dec']
comment_column = tab_sorted['comment']
segm_matched = segm.copy()
segm_matched.keep_labels(label_column)

# Do photometry in every band with SourceCatalog

In [91]:
import glob
img_path_list = glob.glob(img_dir + 'hlsp*.fits')
img_path_list.sort()
img_path_list

['../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f115w_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f150w_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f200w_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f277w_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f356w_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f410m_dr0.5_i2d.fits',
 '../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f444w_dr0.5_i2d.fits']

In [92]:
from utils.zeropoint import calc_zpt
PIXAR_SR = 2.11539874851881E-14
ZERO_POINT =  -6.10 -2.5 * np.log10(PIXAR_SR)
print(f'ZERO_POINT: {ZERO_POINT}')
def flux_to_mag(flux, zero_point=ZERO_POINT):
    return -2.5 * np.log10(flux) + zero_point

ZERO_POINT: 28.086519392283982


In [93]:
#check duplicate
from collections import Counter
counter = Counter(label_column)
duplicates = [k for k, v in counter.items() if v > 1]
if len(duplicates) > 0:
    print(f'Duplicate ID: {duplicates}')
    raise ValueError('Duplicate ID')
else:
    print('No duplicate ID')

No duplicate ID


In [94]:
tab_phot = Table([id_column, ra_column, dec_column, comment_column], names=['ID', 'RA', 'Dec', 'comment'])
band_list = []
for i,img_path in enumerate(img_path_list):
    sci = fits.getdata(img_path, hdu='SCI_BKSUB')
    err = fits.getdata(img_path, hdu='ERR')
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    key_tab = f'KronPhot_{band}_mag'
    cat_sci = SourceCatalog(sci, segm_matched, error=err)
    flux_list = cat_sci.kron_flux
    fluxerr_list = cat_sci.kron_fluxerr
    re_list = cat_sci.fluxfrac_radius(0.5)
    x, y = cat_sci.xcentroid, cat_sci.ycentroid
    mag_list = []
    for j in range(len(flux_list)):
        x_pix, y_pix, flux = x[j], y[j], flux_list[j]
        zpt = calc_zpt(x_pix, y_pix, 'CR0')[i]
        mag = flux_to_mag(flux, zero_point= zpt)
        mag_list.append(mag)
    tab_phot.add_columns([mag_list, flux_list, fluxerr_list, re_list], names=[key_tab, key_tab.replace('mag', 'flux'), key_tab.replace('mag', 'fluxerr'), key_tab.replace('mag', 'Re')])

In [95]:
tab_phot.write(prep_dir + 'kronphot.csv', format='ascii.csv', overwrite=True)
tab_phot[0]

ID,RA,Dec,comment,KronPhot_F115W_mag,KronPhot_F115W_flux,KronPhot_F115W_fluxerr,KronPhot_F115W_Re,KronPhot_F150W_mag,KronPhot_F150W_flux,KronPhot_F150W_fluxerr,KronPhot_F150W_Re,KronPhot_F200W_mag,KronPhot_F200W_flux,KronPhot_F200W_fluxerr,KronPhot_F200W_Re,KronPhot_F277W_mag,KronPhot_F277W_flux,KronPhot_F277W_fluxerr,KronPhot_F277W_Re,KronPhot_F356W_mag,KronPhot_F356W_flux,KronPhot_F356W_fluxerr,KronPhot_F356W_Re,KronPhot_F410M_mag,KronPhot_F410M_flux,KronPhot_F410M_fluxerr,KronPhot_F410M_Re,KronPhot_F444W_mag,KronPhot_F444W_flux,KronPhot_F444W_fluxerr,KronPhot_F444W_Re
,,,,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix,,,,pix
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
35055,214.845323,52.799402,--,27.33206220914366,2.0034701364250433,0.33934832644644014,2.0196201129870546,26.956867760124243,2.830483664949673,0.3656981499872501,2.7943658282267334,26.82717681606334,3.1896059330007,0.3748660098783123,2.9400356791596054,26.879959219035484,3.038253936724298,0.31511939132067557,3.232427226454702,27.15206226249869,2.364737028727041,0.2224313373053323,3.6123770659312653,27.408260650806,1.8676844226823595,0.20028403753456192,3.912943546378243,27.24473857534574,2.1712624753586542,0.1963323271018021,5.0805725478008625


# Write to FITS file for GALFIT preparation

#### IMGLIST Extension

In [96]:
pregalfit_path = prep_dir + 'pregalfit.fits'

# Create an IMGLIST extension
hdr_img = fits.Header()
hdr_img['EXTNAME'] = 'IMGLIST'

tab_img = Table()
band_list = []
for img_path in img_path_list:
    print(img_path)
    hdr_pri = fits.getheader(img_path, extname='PRIMARY')
    band = hdr_pri['FILTER'].split(' ')[0]
    band_list.append(band)
tab_img.add_columns([band_list,img_path_list], names=['band', 'img_path'])
fits.writeto(pregalfit_path, tab_img.as_array(), header=hdr_img, overwrite=True)

../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f115w_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f150w_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f200w_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f277w_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f356w_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f410m_dr0.5_i2d.fits
../data/image/nircam6/hlsp_ceers_jwst_nircam_nircam6_f444w_dr0.5_i2d.fits


#### INIPARAM Extension

In [97]:
srccat = SourceCatalog(image_merged_conv, segm_matched, wcs=w, convolved_data=image_merged_conv, progress_bar=True)
srctab = srccat.to_table(columns = ['label', 'xcentroid', 'ycentroid','orientation','elongation','semimajor_sigma','semiminor_sigma'])
srctab['label'] = srctab['label'].astype(str)
kron_rad = srccat.kron_radius*np.sqrt(srccat.semimajor_sigma.value*srccat.semiminor_sigma.value)
srctab.add_column(kron_rad, name='kron_radius')
tab_final = hstack([tab_phot,srctab])
tab_final = tab_final[tab_final.argsort('ID')]

kron_radius:   0%|          | 0/89 [00:00<?, ?it/s]

In [98]:
# Create a INIPARAM extension
hdr_ini = fits.Header()
hdr_ini['EXTNAME'] = 'INIPARAM'
fits.append(pregalfit_path, tab_final.as_array(), header=hdr_ini, overwrite=True)

hdr_segm = fits.Header()
hdr_segm['EXTNAME'] = 'SEGMAP'
fits.append(pregalfit_path, segm.data, header=hdr_segm, overwrite=True)